In [11]:
import ee
from ee import image
import folium
from folium import features
from folium.plugins import GroupedLayerControl
import os
import webbrowser

# Triggering authentification to earth engine services
# Run in terminal: earthengine authenticate

# initializing the earth engine library (always keep this on)
ee.Initialize()

# ##### Define a method for displaying Earth Engine image tiles to folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  layer = folium.raster_layers.TileLayer(
      tiles = map_id_dict['tile_fetcher'].url_format,
      attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name = name,
      overlay = True,
      control = True
  )
  layer.add_to(self)
  return layer
  
folium.Map.add_ee_layer = add_ee_layer

# Buffer/Circular AOI
aoi = ee.Geometry.Point([2.28, 36.615]).buffer(8000)

# Working with specific satellite imagery ID (sentinel-2)
image_satellite1 = ee.Image('COPERNICUS/S2_SR/20220218T102949_20220218T103126_T31SDA')
image_satellite2 = ee.Image('COPERNICUS/S2_SR/20230107T104329_20230107T104329_T31SDA')

# clipping satellite images to an Area Of Interest (a buffer circle in this case)
image_satellite1 = image_satellite1.clip(aoi).divide(10000)
image_satellite2 = image_satellite2.clip(aoi).divide(10000)

# Sattelite image visual parameters
image_params = {
  'bands': ['B4',  'B3',  'B2'],
  'min': 0,
  'max': 0.35,
  'gamma': 0.9
}

#################### MAIN MAP ####################
# setting up the main map for the project
m = folium.Map(location = [36.62, 2.3], tiles=None, zoom_start = 12, control_scale = True)

# ########## Primary basemaps (Victor data):
basemap1 = folium.TileLayer('cartodbdark_matter', name='Dark Matter').add_to(m)
basemap2 = folium.TileLayer('openstreetmap', name='Open Street Map').add_to(m)

#################### SPATIAL FEATURES LAYERS ####################

# #### MARKERS features layers
blue_marker_group = folium.FeatureGroup(
  name='Blue Marker',
  overlay=True,
  control=True,
  show=True
).add_to(m)

# Individual merkers (Not from a geojson file)
BLUE_MARKER = folium.Marker(
  location=[36.63, 2.26],
  popup= None,
  tooltip="<h4>Blue Marker.</h4>"
).add_to(blue_marker_group)

#################### GEE RASTER LAYERS ####################
# adding the satellite images as layers and display them on the map + on layer panel
image_sat_1_layer = m.add_ee_layer(image_satellite1, image_params, 'Image Satellite 1')
image_sat_2_layer = m.add_ee_layer(image_satellite2, image_params, 'Image Satellite 2')

#################### Layer controller ####################
folium.LayerControl(collapsed=False).add_to(m)

#################### LAYER GROUPS ####################
GroupedLayerControl(
    groups={
      '--------MARKERS LAYER--------': [blue_marker_group],
      '--GOOLE EARTH ENGINE LAYERS--': [image_sat_1_layer, image_sat_2_layer]
    },
    exclusive_groups=False,
    collapsed=False
).add_to(m)

# save map as html file
m.save('map.html')
# Opening the map file in default browser on execution
webbrowser.open('map.html')

True